# Import

In [3]:
import requests
import re
from bs4 import BeautifulSoup

In [4]:
import pandas as pd

In [5]:
from tqdm import tqdm

In [6]:
import time

In [7]:
import warnings
warnings.filterwarnings('ignore')


# Drafts

In [21]:
def parse_bazaraki_property(html: str, page_url: str) -> dict:
    soup = BeautifulSoup(html, 'html.parser')
    
    def get_text_by_label(label):
        tag = soup.find('span', text=label)
        if tag:
            value_span = tag.find_next('span', class_='value-chars')
            if value_span:
                return value_span.get_text(strip=True)
        return None

    data = {
        'Link': page_url,
        'District': soup.find('a', class_='announcement__location').get_text(strip=True) if soup.find('a', class_='announcement__location') else None,
        'CountOfRooms': get_text_by_label('Bedrooms:'),
        'Floor': get_text_by_label('Floor:'),
        'Square': get_text_by_label('Property area:'),
        'Furnishing': get_text_by_label('Furnishing:'),
        'Parking': get_text_by_label('Parking:'),
        'Balcony': 'Balcony' in soup.get_text(),  # fuzzy match
        'AC': get_text_by_label('Air conditioning:'),
        'EnergyEfficiency': get_text_by_label('Energy Efficiency:'),
        'DistanceToJob': '100 meters from the sea' in soup.get_text(),  # example heuristic
        'DistanceToBeach': '100 meters from the sea' in soup.get_text(),
        'DoubleGlazedWindows': 'double glazed' in soup.get_text().lower(),
        'Elevator': 'elevator' in soup.get_text().lower(),
        'WiFi': 'wifi' in soup.get_text().lower() or 'wi-fi' in soup.get_text().lower(),
        'SoundProofing': 'soundproof' in soup.get_text().lower(),
    }

    # Extracting price
    price_block = soup.select_one('.announcement-price__cost')
    if price_block:
        price_text = price_block.get_text(strip=True)
        price_match = re.search(r'€\s?([\d,.]+)', price_text)
        data['Price'] = price_match.group(1).replace(',', '') if price_match else None
    else:
        data['Price'] = None

    return data

In [22]:
# Example usage
url = 'https://www.bazaraki.com/adv/5708036_2-bedroom-apartment-to-rent/'
headers = {'User-Agent': 'Mozilla/5.0'}
response = requests.get(url, headers=headers)
listing_data = parse_bazaraki_property(response.text, url)
print(listing_data)

{'Link': 'https://www.bazaraki.com/adv/5708036_2-bedroom-apartment-to-rent/', 'District': 'Limassol, Limassol - Neapolis', 'CountOfRooms': '1', 'Floor': '2nd', 'Square': '2nd', 'Furnishing': 'Full, all rooms', 'Parking': 'Uncovered', 'Balcony': True, 'AC': 'Full, all rooms', 'EnergyEfficiency': 'B', 'DistanceToJob': True, 'DistanceToBeach': True, 'DoubleGlazedWindows': False, 'Elevator': False, 'WiFi': False, 'SoundProofing': False, 'Price': '1.300'}


/var/folders/0d/nn90v_dn4y56s_xnhsv23lf00000gn/T/ipykernel_14145/1141320045.py:5: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  tag = soup.find('span', text=label)


# Links

In [8]:
link_text = f"""
             BazarakiBot, [2 мар. 2025 в 13:05]
Subscription saved to: https://www.bazaraki.com/real-estate-to-rent/apartments-flats/area_min---50/furnishing---1/number-of-bedrooms---2/?ordering=cheapest&price_max=1750&polygon=%7Br%7BrEaj%7BgEbiHx%7BD%7BjAikSdhE%7DvAwoJseWs%7BK~%7BEuW%7CkKrsApoG

BazarakiBot, [2 мар. 2025 в 13:07]
€1350 2-bedroom apartment to rent
Limassol, Limassol - Petrou Kai Pavlou
https://www.bazaraki.com/adv/5645424_2-bedroom-apartment-to-rent/

BazarakiBot, [2 мар. 2025 в 13:07]
€1550 2-bedroom apartment to rent
Limassol, Ypsonas
https://www.bazaraki.com/adv/5590302_2-bedroom-apartment-to-rent/

BazarakiBot, [2 мар. 2025 в 13:07]
€1590 2-bedroom apartment to rent
Limassol, Limassol - Zakaki
https://www.bazaraki.com/adv/5660339_2-bedroom-apartment-to-rent/

BazarakiBot, [2 мар. 2025 в 13:07]
€1650 2-bedroom apartment to rent
Limassol, Limassol - Agios Nektarios
https://www.bazaraki.com/adv/5619778_2-bedroom-apartment-to-rent/

BazarakiBot, [2 мар. 2025 в 13:07]
€1700 2-bedroom apartment to rent
Limassol, Germasogeia Tourist Area
https://www.bazaraki.com/adv/5552662_1-bedroom-penthouse-to-rent/

BazarakiBot, [2 мар. 2025 в 13:07]
€1750 2-bedroom apartment to rent
Limassol, Limassol - Katholiki
https://www.bazaraki.com/adv/5461258_2-bedroom-apartment-to-rent/

BazarakiBot, [2 мар. 2025 в 13:07]
€650 2-bedroom apartment to rent
Limassol, Limassol - Agios Georgios
https://www.bazaraki.com/adv/5679527_2-bedroom-apartment-to-rent/

BazarakiBot, [2 мар. 2025 в 13:07]
€950 2-bedroom apartment to rent
Limassol, Kolossi -Agios Loukas
https://www.bazaraki.com/adv/5606152_2-bedroom-apartment-to-rent/

BazarakiBot, [2 мар. 2025 в 13:07]
€1000 2-bedroom apartment to rent
Limassol, Limassol - Agios Spyridon
https://www.bazaraki.com/adv/5678254_2-bedroom-apartment-to-rent/

BazarakiBot, [2 мар. 2025 в 13:07]
€1000 2-bedroom apartment to rent
Limassol, Limassol - Linopetra
https://www.bazaraki.com/adv/5674188_2-bedroom-apartment-to-rent/

BazarakiBot, [2 мар. 2025 в 13:07]
€1000 2-bedroom apartment to rent
Limassol, Limassol - Zakaki
https://www.bazaraki.com/adv/5650999_2-bedroom-apartment-to-rent/

BazarakiBot, [2 мар. 2025 в 13:07]
€1000 2-bedroom apartment to rent
Limassol, Limassol - Agios Ioannis
https://www.bazaraki.com/adv/5635812_2-bedroom-apartment-to-rent/

BazarakiBot, [2 мар. 2025 в 13:07]
€1100 2-bedroom apartment to rent
Limassol, Agios Athanasios
https://www.bazaraki.com/adv/5624938_2-bedroom-apartment-to-rent/

BazarakiBot, [2 мар. 2025 в 13:07]
€1100 2-bedroom apartment to rent
Limassol, Erimi
https://www.bazaraki.com/adv/5643367_2-bedroom-apartment-to-rent/

BazarakiBot, [2 мар. 2025 в 13:07]
€1100 2-bedroom apartment to rent
Limassol, Limassol - Omonia
https://www.bazaraki.com/adv/5642861_2-bedroom-apartment-to-rent/

BazarakiBot, [2 мар. 2025 в 13:07]
€1150 2-bedroom apartment to rent
Limassol, Limassol - Mesa Geitonia
https://www.bazaraki.com/adv/5670002_2-bedroom-apartment-to-rent/

BazarakiBot, [2 мар. 2025 в 13:07]
€1200 2-bedroom apartment to rent
Limassol, Agios Tychon Tourist Area
https://www.bazaraki.com/adv/5664359_2-bedroom-apartment-to-rent/

BazarakiBot, [2 мар. 2025 в 13:07]
€1200 2-bedroom apartment to rent
Limassol, Germasogeia Tourist Area
https://www.bazaraki.com/adv/5678301_2-bedroom-apartment-to-rent/

BazarakiBot, [2 мар. 2025 в 13:07]
€1200 2-bedroom apartment to rent
Limassol, Potamos Germasogeias
https://www.bazaraki.com/adv/5272205_2-bedroom-apartment-to-rent/

BazarakiBot, [2 мар. 2025 в 13:07]
€1200 2-bedroom apartment to rent
Limassol, Limassol - Petrou Kai Pavlou
https://www.bazaraki.com/adv/5677113_2-bedroom-apartment-to-rent/

BazarakiBot, [2 мар. 2025 в 13:07]
€1200 2-bedroom apartment to rent
Limassol, Limassol - Zakaki
https://www.bazaraki.com/adv/5636325_2-bedroom-penthouse-to-rent/

BazarakiBot, [2 мар. 2025 в 13:07]
€1200 2-bedroom apartment to rent
Limassol, Limassol - Agios Ioannis
https://www.bazaraki.com/adv/5601682_2-bedroom-apartment-to-rent/

BazarakiBot, [2 мар. 2025 в 13:07]
€1200 2-bedroom apartment to rent
Limassol, Limassol - Kapsalos
https://www.bazaraki.com/adv/5666842_2-bedroom-apartment-to-rent/

BazarakiBot, [2 мар. 2025 в 13:07]
€1200 2-bedroom apartment to rent
Limassol, Limassol - Agia Zoni
https://www.bazaraki.com/adv/5635691_2-bedroom-apartment-to-rent/

BazarakiBot, [2 мар. 2025 в 13:07]
€1200 2-bedroom apartment to rent
Limassol, Limassol - Linopetra
https://www.bazaraki.com/adv/5665425_2-bedroom-apartment-to-rent/

BazarakiBot, [2 мар. 2025 в 13:07]
€1200 2-bedroom apartment to rent
Limassol, Limassol - Linopetra
https://www.bazaraki.com/adv/5625342_2-bedroom-apartment-to-rent/

BazarakiBot, [2 мар. 2025 в 13:07]
€1200 2-bedroom apartment to rent
Limassol, Ypsonas
https://www.bazaraki.com/adv/5593137_2-bedroom-apartment-to-rent/

BazarakiBot, [2 мар. 2025 в 13:07]
€1250 2-bedroom apartment to rent
Limassol, Germasogeia Tourist Area
https://www.bazaraki.com/adv/5664143_2-bedroom-apartment-to-rent/

BazarakiBot, [2 мар. 2025 в 13:07]
€1250 2-bedroom apartment to rent
Limassol, Limassol - Katholiki
https://www.bazaraki.com/adv/5613596_1-bedroom-apartment-to-rent/

BazarakiBot, [2 мар. 2025 в 13:07]
€1250 2-bedroom apartment to rent
Limassol, Germasogeia Tourist Area
https://www.bazaraki.com/adv/5486591_2-bedroom-apartment-to-rent/

BazarakiBot, [2 мар. 2025 в 13:07]
€1300 2-bedroom apartment to rent
Limassol, Agios Tychon Tourist Area
https://www.bazaraki.com/adv/5631055_2-bedroom-apartment-to-rent/

BazarakiBot, [2 мар. 2025 в 13:07]
€1300 2-bedroom apartment to rent
Limassol, Agios Tychon Tourist Area
https://www.bazaraki.com/adv/5657823_2-bedroom-apartment-to-rent/

BazarakiBot, [2 мар. 2025 в 13:07]
€1300 2-bedroom apartment to rent
Limassol, Germasogeia Tourist Area
https://www.bazaraki.com/adv/5645042_2-bedroom-apartment-to-rent/

BazarakiBot, [2 мар. 2025 в 13:07]
€1300 2-bedroom apartment to rent
Limassol, Limassol - Mesa Geitonia
https://www.bazaraki.com/adv/5670779_2-bedroom-apartment-to-rent/

BazarakiBot, [2 мар. 2025 в 13:07]
€1300 2-bedroom apartment to rent
Limassol, Limassol - Mesa Geitonia
https://www.bazaraki.com/adv/5673259_2-bedroom-apartment-to-rent/

BazarakiBot, [2 мар. 2025 в 13:07]
€1300 2-bedroom penthouse to rent
Limassol, Limassol - Mesa Geitonia
https://www.bazaraki.com/adv/5671175_2-bedroom-penthouse-to-rent/

BazarakiBot, [2 мар. 2025 в 13:07]
€1300 2-bedroom apartment to rent
Limassol, Limassol - Tsirion
https://www.bazaraki.com/adv/5621114_2-bedroom-apartment-to-rent/

BazarakiBot, [2 мар. 2025 в 13:07]
€1300 2-bedroom apartment to rent
Limassol, Germasogeia
https://www.bazaraki.com/adv/5669603_2-bedroom-apartment-to-rent/

BazarakiBot, [2 мар. 2025 в 13:07]
€1300 2-bedroom apartment to rent
Limassol, Limassol - Agios Georgios
https://www.bazaraki.com/adv/5669159_2-bedroom-apartment-to-rent/

BazarakiBot, [2 мар. 2025 в 13:07]
€1300 2-bedroom apartment to rent
Limassol, Limassol - Agios Spyridon
https://www.bazaraki.com/adv/5558862_2-bedroom-apartment-to-rent/

BazarakiBot, [2 мар. 2025 в 13:07]
€1300 2-bedroom apartment to rent
Limassol, Limassol - Agios Nicolaos
https://www.bazaraki.com/adv/5609635_2-bedroom-apartment-to-rent/

BazarakiBot, [2 мар. 2025 в 13:07]
€1300 2-bedroom apartment to rent
Limassol, Limassol - Kapsalos
https://www.bazaraki.com/adv/5660681_2-bedroom-apartment-to-rent/

BazarakiBot, [2 мар. 2025 в 13:07]
€1300 2-bedroom apartment to rent
Limassol, Limassol - Agios Georgios
https://www.bazaraki.com/adv/5660563_2-bedroom-apartment-to-rent/

BazarakiBot, [2 мар. 2025 в 13:07]
€1300 2-bedroom apartment to rent
Limassol, Polemidia Kato
https://www.bazaraki.com/adv/5610072_2-bedroom-apartment-to-rent/

BazarakiBot, [2 мар. 2025 в 13:07]
€1300 2-bedroom apartment to rent
Limassol, Agios Ioannis
https://www.bazaraki.com/adv/5650676_2-bedroom-apartment-to-rent/

BazarakiBot, [2 мар. 2025 в 13:07]
€1300 2-bedroom apartment to rent
Limassol, Limassol - Kapsalos
https://www.bazaraki.com/adv/5570765_2-bedroom-apartment-to-rent/

BazarakiBot, [2 мар. 2025 в 13:07]
€1300 2-bedroom apartment to rent
Limassol, Polemidia - Apostolos Varnavas
https://www.bazaraki.com/adv/5537810_2-bedroom-apartment-to-rent/

BazarakiBot, [2 мар. 2025 в 13:07]
€1350 2-bedroom apartment to rent
Limassol, Ypsonas
https://www.bazaraki.com/adv/5590310_2-bedroom-apartment-to-rent/

BazarakiBot, [2 мар. 2025 в 13:07]
€1350 2-bedroom apartment to rent
Limassol, Limassol - Zakaki
https://www.bazaraki.com/adv/5675339_2-bedroom-apartment-to-rent/

BazarakiBot, [2 мар. 2025 в 13:07]
€1350 2-bedroom penthouse to rent
Limassol, Limassol - Zakaki
https://www.bazaraki.com/adv/5673782_2-bedroom-penthouse-to-rent/

BazarakiBot, [2 мар. 2025 в 13:07]
€1350 2-bedroom apartment to rent
Limassol, Historical Center
https://www.bazaraki.com/adv/5657706_2-bedroom-apartment-to-rent/

BazarakiBot, [2 мар. 2025 в 13:07]
€1350 2-bedroom apartment to rent
Limassol, Ypsonas
https://www.bazaraki.com/adv/5672977_2-bedroom-apartment-to-rent/

BazarakiBot, [2 мар. 2025 в 13:07]
€1350 2-bedroom apartment to rent
Limassol, Limassol - Agios Nicolaos
https://www.bazaraki.com/adv/5669144_2-bedroom-apartment-to-rent/

BazarakiBot, [2 мар. 2025 в 13:07]
€1350 2-bedroom apartment to rent
Limassol, Ypsonas
https://www.bazaraki.com/adv/5638723_2-bedroom-apartment-to-rent/

BazarakiBot, [2 мар. 2025 в 13:07]
€1350 2-bedroom apartment to rent
Limassol, Polemidia Kato
https://www.bazaraki.com/adv/5664577_2-bedroom-apartment-to-rent/

BazarakiBot, [2 мар. 2025 в 13:07]
€1350 2-bedroom apartment to rent
Limassol, Limassol - Agios Spyridon
https://www.bazaraki.com/adv/5642123_2-bedroom-apartment-to-rent/

BazarakiBot, [2 мар. 2025 в 13:07]
€1350 2-bedroom apartment to rent
Limassol, Germasogeia
https://www.bazaraki.com/adv/5634156_2-bedroom-apartment-to-rent/

BazarakiBot, [2 мар. 2025 в 13:07]
€1350 2-bedroom apartment to rent
Limassol, Historical Center
https://www.bazaraki.com/adv/5640601_2-bedroom-apartment-to-rent/

BazarakiBot, [2 мар. 2025 в 13:07]
€1350 2-bedroom apartment to rent
Limassol, Polemidia Kato
https://www.bazaraki.com/adv/5627688_2-bedroom-apartment-to-rent/

BazarakiBot, [2 мар. 2025 в 17:14]
€1350 2-bedroom apartment to rent
Limassol, Ypsonas
https://www.bazaraki.com/adv/5651041_2-bedroom-apartment-to-rent/

BazarakiBot, [2 мар. 2025 в 23:50]
€1350 2-bedroom apartment to rent
Limassol, Germasogeia Tourist Area
https://www.bazaraki.com/adv/5644708_2-bedroom-apartment-to-rent/

BazarakiBot, [3 мар. 2025 в 11:43]
€1350 2-bedroom apartment to rent
Limassol, Limassol - Tsiflikoudia
https://www.bazaraki.com/adv/5633394_2-bedroom-apartment-to-rent/

BazarakiBot, [3 мар. 2025 в 21:51]
€1365 2-bedroom apartment to rent
Limassol, Limassol - Mesa Geitonia
https://www.bazaraki.com/adv/5651432_2-bedroom-apartment-to-rent/

BazarakiBot, [4 мар. 2025 в 00:11]
€1400 2-bedroom apartment to rent
Limassol, Limassol - Petrou Kai Pavlou
https://www.bazaraki.com/adv/5514212_2-bedroom-apartment-to-rent/

BazarakiBot, [4 мар. 2025 в 09:42]
€1350 2-bedroom apartment to rent
Limassol, Germasogeia
https://www.bazaraki.com/adv/5614537_2-bedroom-apartment-to-rent/

BazarakiBot, [4 мар. 2025 в 12:22]
€1100 2-bedroom apartment to rent
Limassol, Limassol - Zakaki
https://www.bazaraki.com/adv/5677160_2-bedroom-apartment-to-rent/

BazarakiBot, [4 мар. 2025 в 14:25]
€1300 2-bedroom apartment to rent
Limassol, Mouttagiaka Tourist Area
https://www.bazaraki.com/adv/5496193_2-bedroom-apartment-to-rent/

BazarakiBot, [4 мар. 2025 в 14:25]
€1350 2-bedroom apartment to rent
Limassol, Polemidia Pano
https://www.bazaraki.com/adv/5682760_2-bedroom-apartment-to-rent/

BazarakiBot, [4 мар. 2025 в 16:55]
€1350 2-bedroom apartment to rent
Limassol, Polemidia Kato
https://www.bazaraki.com/adv/5683377_2-bedroom-apartment-to-rent/

BazarakiBot, [4 мар. 2025 в 17:16]
€1100 2-bedroom apartment to rent
Limassol, Limassol Marina
https://www.bazaraki.com/adv/5683405_2-bedroom-apartment-to-rent/

BazarakiBot, [4 мар. 2025 в 18:17]
€1350 2-bedroom apartment to rent
Limassol, Limassol - Petrou Kai Pavlou
https://www.bazaraki.com/adv/5683648_2-bedroom-apartment-to-rent/

BazarakiBot, [4 мар. 2025 в 18:17]
€1350 2-bedroom apartment to rent
Limassol, Limassol - Agios Nicolaos
https://www.bazaraki.com/adv/5683639_2-bedroom-apartment-to-rent/

BazarakiBot, [4 мар. 2025 в 18:28]
€1350 2-bedroom apartment to rent
Limassol, Limassol - Zakaki
https://www.bazaraki.com/adv/5683655_2-bedroom-apartment-to-rent/

BazarakiBot, [4 мар. 2025 в 21:54]
€1200 2-bedroom apartment to rent
Limassol, Germasogeia
https://www.bazaraki.com/adv/5540812_1-bedroom-apartment-to-rent/

BazarakiBot, [4 мар. 2025 в 22:05]
€1350 2-bedroom apartment to rent
Limassol, Limassol - Agios Nicolaos
https://www.bazaraki.com/adv/5683980_2-bedroom-apartment-to-rent/

BazarakiBot, [4 мар. 2025 в 22:25]
€1350 2-bedroom apartment to rent
Limassol, Historical Center
https://www.bazaraki.com/adv/5684009_2-bedroom-apartment-to-rent/

BazarakiBot, [5 мар. 2025 в 12:46]
€1300 2-bedroom apartment to rent
Limassol, Germasogeia Tourist Area
https://www.bazaraki.com/adv/5684816_2-bedroom-apartment-to-rent/

BazarakiBot, [5 мар. 2025 в 14:34]
€ 1350 ➝ 1250 2-bedroom apartment to rent
Limassol, Ypsonas
https://www.bazaraki.com/adv/5672977_2-bedroom-apartment-to-rent/

BazarakiBot, [5 мар. 2025 в 15:23]
€1350 2-bedroom apartment to rent
Limassol, Limassol - Agios Nicolaos
https://www.bazaraki.com/adv/5685112_2-bedroom-apartment-to-rent/

BazarakiBot, [5 мар. 2025 в 16:59]
€1100 2-bedroom apartment to rent
Limassol, Polemidia Kato
https://www.bazaraki.com/adv/5685131_2-bedroom-apartment-to-rent/

BazarakiBot, [5 мар. 2025 в 17:48]
€1500 2-bedroom apartment to rent
Limassol, Limassol - Zakaki
https://www.bazaraki.com/adv/5607752_2-bedroom-apartment-to-rent/

BazarakiBot, [5 мар. 2025 в 17:58]
€1740 2-bedroom apartment to rent
Limassol, Mouttagiaka Tourist Area
https://www.bazaraki.com/adv/5630099_2-bedroom-apartment-to-rent/

BazarakiBot, [5 мар. 2025 в 18:09]
€ 1650 ➝ 1450 2-bedroom apartment to rent
Limassol, Limassol - Agios Nektarios
https://www.bazaraki.com/adv/5619778_2-bedroom-apartment-to-rent/

BazarakiBot, [6 мар. 2025 в 11:40]
€1250 2-bedroom apartment to rent
Limassol, Limassol - Agios Nektarios
https://www.bazaraki.com/adv/5686561_2-bedroom-apartment-to-rent/

BazarakiBot, [6 мар. 2025 в 12:09]
€1500 2-bedroom apartment to rent
Limassol, Limassol - Neapolis
https://www.bazaraki.com/adv/5686603_2-bedroom-apartment-to-rent/

BazarakiBot, [6 мар. 2025 в 12:09]
€1600 2-bedroom apartment to rent
Limassol, Limassol - Agia Zoni
https://www.bazaraki.com/adv/5686560_2-bedroom-apartment-to-rent/

BazarakiBot, [6 мар. 2025 в 13:10]
€ 1500 ➝ 1450 2-bedroom apartment to rent
Limassol, Limassol - Zakaki
https://www.bazaraki.com/adv/5607752_2-bedroom-apartment-to-rent/

BazarakiBot, [6 мар. 2025 в 13:10]
€ 1740 ➝ 1690 2-bedroom apartment to rent
Limassol, Mouttagiaka Tourist Area
https://www.bazaraki.com/adv/5630099_2-bedroom-apartment-to-rent/

BazarakiBot, [6 мар. 2025 в 13:46]
€1350 2-bedroom apartment to rent
Limassol, Limassol - Omonia
https://www.bazaraki.com/adv/5397909_2-bedroom-apartment-to-rent/

BazarakiBot, [6 мар. 2025 в 16:42]
€1500 2-bedroom apartment to rent
Limassol, Limassol - Neapolis
https://www.bazaraki.com/adv/5667831_2-bedroom-apartment-to-rent/

BazarakiBot, [6 мар. 2025 в 23:02]
€1350 2-bedroom apartment to rent
Limassol, Limassol - Zakaki
https://www.bazaraki.com/adv/5545760_2-bedroom-apartment-to-rent/

BazarakiBot, [7 мар. 2025 в 11:07]
€1350 2-bedroom apartment to rent
Limassol, Potamos Germasogeias
https://www.bazaraki.com/adv/5640116_2-bedroom-apartment-to-rent/

BazarakiBot, [7 мар. 2025 в 15:06]
€1600 2-bedroom apartment to rent
Limassol, Limassol - Mesa Geitonia
https://www.bazaraki.com/adv/5688850_2-bedroom-apartment-to-rent/

BazarakiBot, [7 мар. 2025 в 16:27]
€ 1300 ➝ 1200 2-bedroom apartment to rent
Limassol, Limassol - Agios Spyridon
https://www.bazaraki.com/adv/5558862_2-bedroom-apartment-to-rent/

BazarakiBot, [7 мар. 2025 в 16:56]
€1350 2-bedroom apartment to rent
Limassol, Limassol - Agia Zoni
https://www.bazaraki.com/adv/5644684_2-bedroom-apartment-to-rent/

BazarakiBot, [7 мар. 2025 в 22:39]
€1350 2-bedroom apartment to rent
Limassol, Polemidia Kato
https://www.bazaraki.com/adv/5689660_2-bedroom-apartment-to-rent/

BazarakiBot, [8 мар. 2025 в 10:24]
€ 1350 ➝ 1300 2-bedroom apartment to rent
Limassol, Limassol - Zakaki
https://www.bazaraki.com/adv/5545760_2-bedroom-apartment-to-rent/

BazarakiBot, [8 мар. 2025 в 12:53]
€1200 2-bedroom apartment to rent
Limassol, Limassol - Apostolos Andreas
https://www.bazaraki.com/adv/5690093_2-bedroom-apartment-to-rent/

BazarakiBot, [8 мар. 2025 в 15:54]
€950 2-bedroom apartment to rent
Limassol, Limassol - Omonia
https://www.bazaraki.com/adv/5690355_2-bedroom-apartment-to-rent/

BazarakiBot, [8 мар. 2025 в 16:44]
€1300 2-bedroom apartment to rent
Limassol, Limassol - Neapolis
https://www.bazaraki.com/adv/5682838_2-bedroom-apartment-to-rent/

BazarakiBot, [8 мар. 2025 в 17:52]
€1000 2-bedroom apartment to rent
Limassol, Germasogeia
https://www.bazaraki.com/adv/5690474_2-bedroom-apartment-to-rent/

BazarakiBot, [9 мар. 2025 в 11:55]
€1200 2-bedroom apartment to rent
Limassol, Limassol - Tsirion
https://www.bazaraki.com/adv/5691074_2-bedroom-apartment-to-rent/

BazarakiBot, [10 мар. 2025 в 09:21]
€1650 2-bedroom apartment to rent
Limassol, Polemidia Kato
https://www.bazaraki.com/adv/5692055_2-bedroom-apartment-to-rent/

BazarakiBot, [10 мар. 2025 в 10:02]
€1100 2-bedroom apartment to rent
Limassol, Polemidia - Apostolos Varnavas
https://www.bazaraki.com/adv/5692081_2-bedroom-apartment-to-rent/

BazarakiBot, [10 мар. 2025 в 10:31]
€1750 2-bedroom apartment to rent
Limassol, Panthea
https://www.bazaraki.com/adv/5692107_2-bedroom-apartment-to-rent/

BazarakiBot, [10 мар. 2025 в 13:04]
€ 1690 ➝ 1600 2-bedroom apartment to rent
Limassol, Mouttagiaka Tourist Area
https://www.bazaraki.com/adv/5630099_2-bedroom-apartment-to-rent/

BazarakiBot, [10 мар. 2025 в 13:04]
€1350 2-bedroom apartment to rent
Limassol, Limassol - Agios Ioannis
https://www.bazaraki.com/adv/5673455_4-bedroom-apartment-to-rent/

BazarakiBot, [10 мар. 2025 в 17:35]
€1300 2-bedroom apartment to rent
Limassol, Potamos Germasogeias
https://www.bazaraki.com/adv/5676627_2-bedroom-apartment-to-rent/

BazarakiBot, [10 мар. 2025 в 18:36]
€ 1590 ➝ 1490 2-bedroom apartment to rent
Limassol, Limassol - Zakaki
https://www.bazaraki.com/adv/5660339_2-bedroom-apartment-to-rent/

BazarakiBot, [10 мар. 2025 в 18:36]
€1200 2-bedroom apartment to rent
Limassol, Limassol - Omonia
https://www.bazaraki.com/adv/5693101_2-bedroom-apartment-to-rent/

BazarakiBot, [10 мар. 2025 в 22:45]
€1200 2-bedroom apartment to rent
Limassol, Limassol - Agios Ioannis
https://www.bazaraki.com/adv/5693505_2-bedroom-apartment-to-rent/

BazarakiBot, [11 мар. 2025 в 08:55]
€ 1300 ➝ 1200 2-bedroom apartment to rent
Limassol, Agios Tychon Tourist Area
https://www.bazaraki.com/adv/5657823_2-bedroom-apartment-to-rent/

BazarakiBot, [11 мар. 2025 в 10:04]
€ 1300 ➝ 1200 2-bedroom apartment to rent
Limassol, Mouttagiaka Tourist Area
https://www.bazaraki.com/adv/5496193_2-bedroom-apartment-to-rent/

BazarakiBot, [11 мар. 2025 в 10:54]
€1350 2-bedroom apartment to rent
Limassol, Limassol - Omonia
https://www.bazaraki.com/adv/5693859_2-bedroom-apartment-to-rent/

BazarakiBot, [11 мар. 2025 в 12:35]
€ 1300 ➝ 1200 2-bedroom apartment to rent
Limassol, Germasogeia Tourist Area
https://www.bazaraki.com/adv/5684816_2-bedroom-apartment-to-rent/

BazarakiBot, [11 мар. 2025 в 13:39]
€1250 2-bedroom apartment to rent
Limassol, Limassol - Zakaki
https://www.bazaraki.com/adv/5693977_2-bedroom-apartment-to-rent/

BazarakiBot, [11 мар. 2025 в 16:11]
€1400 2-bedroom apartment to rent
Limassol, Mesa Geitonia
https://www.bazaraki.com/adv/5598297_2-bedroom-apartment-to-rent/

BazarakiBot, [11 мар. 2025 в 18:16]
€1300 2-bedroom apartment to rent
Limassol, Limassol - Mesa Geitonia
https://www.bazaraki.com/adv/5694782_2-bedroom-apartment-to-rent/

BazarakiBot, [11 мар. 2025 в 20:26]
€1350 2-bedroom apartment to rent
Limassol, Limassol Marina
https://www.bazaraki.com/adv/5651172_2-bedroom-apartment-to-rent/

BazarakiBot, [12 мар. 2025 в 09:28]
€1000 2-bedroom apartment to rent
Limassol, Limassol - Apostolos Andreas
https://www.bazaraki.com/adv/5695480_2-bedroom-apartment-to-rent/

BazarakiBot, [12 мар. 2025 в 10:48]
€1300 2-bedroom apartment to rent
Limassol, Polemidia - Agios Nikolaos
https://www.bazaraki.com/adv/5671166_2-bedroom-apartment-to-rent/

BazarakiBot, [12 мар. 2025 в 15:02]
€1200 2-bedroom apartment to rent
Limassol, Limassol - Agia Zoni
https://www.bazaraki.com/adv/5696037_2-bedroom-apartment-to-rent/

BazarakiBot, [12 мар. 2025 в 22:34]
€1350 2-bedroom apartment to rent
Limassol, Limassol - Agia Napa
https://www.bazaraki.com/adv/5684007_2-bedroom-apartment-to-rent/

BazarakiBot, [12 мар. 2025 в 23:14]
€1300 2-bedroom apartment to rent
Limassol, Limassol - Agios Nektarios
https://www.bazaraki.com/adv/5697264_2-bedroom-apartment-to-rent/

BazarakiBot, [13 мар. 2025 в 10:30]
€1295 2-bedroom apartment to rent
Limassol, Limassol - Agia Fyla
https://www.bazaraki.com/adv/5563133_2-bedroom-apartment-to-rent/

BazarakiBot, [13 мар. 2025 в 12:37]
€1300 2-bedroom penthouse to rent
Limassol, Limassol - Mesa Geitonia
https://www.bazaraki.com/adv/5697715_2-bedroom-penthouse-to-rent/

BazarakiBot, [14 мар. 2025 в 09:26]
€ 1350 ➝ 1300 2-bedroom apartment to rent
Limassol, Historical Center
https://www.bazaraki.com/adv/5684009_2-bedroom-apartment-to-rent/

BazarakiBot, [14 мар. 2025 в 18:04]
€1300 2-bedroom apartment to rent
Limassol, Limassol - Agios Nicolaos
https://www.bazaraki.com/adv/5683886_2-bedroom-apartment-to-rent/

BazarakiBot, [15 мар. 2025 в 18:27]
€ 1300 ➝ 1200 2-bedroom apartment to rent
Limassol, Limassol - Neapolis
https://www.bazaraki.com/adv/5682838_2-bedroom-apartment-to-rent/

BazarakiBot, [16 мар. 2025 в 16:41]
€1200 2-bedroom apartment to rent
Limassol, Limassol - Agia Fyla
https://www.bazaraki.com/adv/5702668_2-bedroom-apartment-to-rent/

BazarakiBot, [17 мар. 2025 в 10:16]
€850 2-bedroom apartment to rent
Limassol, Ypsonas
https://www.bazaraki.com/adv/5703204_2-bedroom-apartment-to-rent/

BazarakiBot, [17 мар. 2025 в 11:35]
€1200 2-bedroom apartment to rent
Limassol, Historical Center
https://www.bazaraki.com/adv/5703433_2-bedroom-apartment-to-rent/

BazarakiBot, [17 мар. 2025 в 11:40]
€1150 2-bedroom apartment to rent
Limassol, Limassol - Neapolis
https://www.bazaraki.com/adv/5703404_2-bedroom-apartment-to-rent/

BazarakiBot, [17 мар. 2025 в 12:06]
€1200 2-bedroom apartment to rent
Limassol, Germasogeia
https://www.bazaraki.com/adv/5517035_2-bedroom-detached-house-to-rent/

BazarakiBot, [17 мар. 2025 в 12:58]
€1300 2-bedroom apartment to rent
Limassol, Mesa Geitonia
https://www.bazaraki.com/adv/5634085_2-bedroom-apartment-to-rent/

BazarakiBot, [17 мар. 2025 в 15:18]
€1300 2-bedroom apartment to rent
Limassol, Limassol - Katholiki
https://www.bazaraki.com/adv/5697467_2-bedroom-apartment-to-rent/

BazarakiBot, [17 мар. 2025 в 19:15]
€1300 2-bedroom apartment to rent
Limassol, Germasogeia Tourist Area
https://www.bazaraki.com/adv/5705332_2-bedroom-apartment-to-rent/

BazarakiBot, [17 мар. 2025 в 19:24]
€1350 2-bedroom apartment to rent
Limassol, Kolossi -Agios Loukas
https://www.bazaraki.com/adv/5698300_2-bedroom-apartment-to-rent/

BazarakiBot, [17 мар. 2025 в 20:26]
€1300 2-bedroom apartment to rent
Limassol, Mesa Geitonia
https://www.bazaraki.com/adv/5705448_2-bedroom-apartment-to-rent/

BazarakiBot, [17 мар. 2025 в 20:46]
€ 1350 ➝ 1300 2-bedroom apartment to rent
Limassol, Historical Center
https://www.bazaraki.com/adv/5657706_2-bedroom-apartment-to-rent/

BazarakiBot, [17 мар. 2025 в 21:26]
€ 1300 ➝ 1200 2-bedroom apartment to rent
Limassol, Limassol - Agios Georgios
https://www.bazaraki.com/adv/5669159_2-bedroom-apartment-to-rent/

BazarakiBot, [18 мар. 2025 в 14:06]
€ 1200 ➝ 1150 2-bedroom apartment to rent
Limassol, Limassol - Apostolos Andreas
https://www.bazaraki.com/adv/5690093_2-bedroom-apartment-to-rent/

BazarakiBot, [18 мар. 2025 в 18:22]
€1200 2-bedroom apartment to rent
Limassol, Historical Center
https://www.bazaraki.com/adv/5604355_2-bedroom-apartment-to-rent/

BazarakiBot, [19 мар. 2025 в 10:15]
€1200 2-bedroom apartment to rent
Limassol, Germasogeia
https://www.bazaraki.com/adv/5708926_2-bedroom-apartment-to-rent/

BazarakiBot, [19 мар. 2025 в 12:07]
€1400 2-bedroom apartment to rent
Limassol, Limassol - Zakaki
https://www.bazaraki.com/adv/5705434_2-bedroom-apartment-to-rent/

BazarakiBot, [19 мар. 2025 в 13:08]
€1400 2-bedroom apartment to rent
Limassol, Limassol - Katholiki
https://www.bazaraki.com/adv/5707844_2-bedroom-apartment-to-rent/

BazarakiBot, [19 мар. 2025 в 15:17]
€1400 2-bedroom apartment to rent
Limassol, Limassol - Neapolis
https://www.bazaraki.com/adv/5708036_2-bedroom-apartment-to-rent/

BazarakiBot, [19 мар. 2025 в 15:39]
€1400 2-bedroom apartment to rent
Limassol, Limassol - Neapolis
https://www.bazaraki.com/adv/5684644_2-bedroom-apartment-to-rent/

BazarakiBot, [19 мар. 2025 в 18:29]
€1400 2-bedroom apartment to rent
Limassol, Limassol - Katholiki
https://www.bazaraki.com/adv/5704773_2-bedroom-apartment-to-rent/

BazarakiBot, [19 мар. 2025 в 19:10]
€1300 2-bedroom apartment to rent
Limassol, Mesa Geitonia
https://www.bazaraki.com/adv/5708561_2-bedroom-apartment-to-rent/

BazarakiBot, [19 мар. 2025 в 19:19]
€1400 2-bedroom apartment to rent
Limassol, Limassol - Katholiki
https://www.bazaraki.com/adv/5710260_2-bedroom-apartment-to-rent/

BazarakiBot, [20 мар. 2025 в 11:49]
€1350 2-bedroom apartment to rent
Limassol, Limassol - Agios Spyridon
https://www.bazaraki.com/adv/5700476_2-bedroom-apartment-to-rent/

BazarakiBot, [20 мар. 2025 в 13:18]
€ 1600 ➝ 1580 2-bedroom apartment to rent
Limassol, Mouttagiaka Tourist Area
https://www.bazaraki.com/adv/5630099_2-bedroom-apartment-to-rent/

BazarakiBot, [20 мар. 2025 в 13:18]
€1350 2-bedroom apartment to rent
Limassol, Limassol - Zakaki
https://www.bazaraki.com/adv/5589210_2-bedroom-apartment-to-rent/

BazarakiBot, [20 мар. 2025 в 17:27]
€1400 2-bedroom penthouse to rent
Limassol, Limassol - Apostolos Andreas
https://www.bazaraki.com/adv/5703109_2-bedroom-penthouse-to-rent/

BazarakiBot, [20 мар. 2025 в 18:37]
€1300 2-bedroom apartment to rent
Limassol, Limassol - Katholiki
https://www.bazaraki.com/adv/5696839_2-bedroom-apartment-to-rent/

BazarakiBot, [20 мар. 2025 в 21:21]
€1200 2-bedroom apartment to rent
Limassol, Polemidia Pano
https://www.bazaraki.com/adv/5712047_2-bedroom-apartment-to-rent/

BazarakiBot, [21 мар. 2025 в 08:28]
€ 1500 ➝ 1300 2-bedroom apartment to rent
Limassol, Limassol - Neapolis
https://www.bazaraki.com/adv/5667831_2-bedroom-apartment-to-rent/

BazarakiBot, [21 мар. 2025 в 11:07]
€1600 2-bedroom apartment to rent
Limassol, Agios Athanasios
https://www.bazaraki.com/adv/5492010_2-bedroom-apartment-to-rent/

BazarakiBot, [21 мар. 2025 в 12:37]
€1300 2-bedroom apartment to rent
Limassol, Mouttagiaka Tourist Area
https://www.bazaraki.com/adv/5712507_2-bedroom-apartment-to-rent/

BazarakiBot, [21 мар. 2025 в 16:08]
€1700 2-bedroom apartment to rent
Limassol, Limassol - Zakaki
https://www.bazaraki.com/adv/5704866_2-bedroom-apartment-to-rent/

BazarakiBot, [21 мар. 2025 в 18:01]
€1200 2-bedroom apartment to rent
Limassol, Limassol - Agios Georgios
https://www.bazaraki.com/adv/5713189_2-bedroom-apartment-to-rent/

BazarakiBot, [21 мар. 2025 в 18:10]
€1300 2-bedroom apartment to rent
Limassol, Germasogeia Tourist Area
https://www.bazaraki.com/adv/5712790_2-bedroom-apartment-to-rent/

BazarakiBot, [21 мар. 2025 в 20:51]
€1350 2-bedroom penthouse to rent
Limassol, Episkopi Lemesou
https://www.bazaraki.com/adv/5620506_3-bedroom-apartment-to-rent/

BazarakiBot, [22 мар. 2025 в 05:39]
€1400 2-bedroom apartment to rent
Limassol, Limassol - Mesa Geitonia
https://www.bazaraki.com/adv/5532017_3-bedroom-apartment-to-rent/

BazarakiBot, [22 мар. 2025 в 09:39]
€1200 2-bedroom apartment to rent
Limassol, Limassol - Omonia
https://www.bazaraki.com/adv/5713817_2-bedroom-apartment-to-rent/

BazarakiBot, [22 мар. 2025 в 10:29]
€ 1750 ➝ 1700 2-bedroom apartment to rent
Limassol, Limassol - Katholiki
https://www.bazaraki.com/adv/5461258_2-bedroom-apartment-to-rent/

BazarakiBot, [22 мар. 2025 в 19:10]
€ 1700 ➝ 1650 2-bedroom apartment to rent
Limassol, Limassol - Katholiki
https://www.bazaraki.com/adv/5461258_2-bedroom-apartment-to-rent/

BazarakiBot, [23 мар. 2025 в 13:16]
€1400 2-bedroom penthouse to rent
Limassol, Limassol - Apostolos Andreas
https://www.bazaraki.com/adv/5711210_2-bedroom-penthouse-to-rent/

BazarakiBot, [23 мар. 2025 в 13:29]
€1700 2-bedroom penthouse to rent
Limassol, Germasogeia
https://www.bazaraki.com/adv/5715425_2-bedroom-penthouse-to-rent/

BazarakiBot, [23 мар. 2025 в 19:05]
€1400 2-bedroom apartment to rent
Limassol, Limassol - Agia Fyla
https://www.bazaraki.com/adv/5715840_2-bedroom-apartment-to-rent/

BazarakiBot, [23 мар. 2025 в 22:36]
€1350 2-bedroom apartment to rent
Limassol, Germasogeia Tourist Area
https://www.bazaraki.com/adv/5716075_2-bedroom-apartment-to-rent/

BazarakiBot, [24 мар. 2025 в 11:31]
€1400 2-bedroom apartment to rent
Limassol, Limassol - Mesa Geitonia
https://www.bazaraki.com/adv/5712375_2-bedroom-apartment-to-rent/

BazarakiBot, [24 мар. 2025 в 12:33]
€1300 2-bedroom apartment to rent
Limassol, Limassol - Neapolis
https://www.bazaraki.com/adv/5685682_2-bedroom-apartment-to-rent/

BazarakiBot, [24 мар. 2025 в 13:14]
€ 1700 ➝ 1600 2-bedroom apartment to rent
Limassol, Limassol - Zakaki
https://www.bazaraki.com/adv/5704866_2-bedroom-apartment-to-rent/

BazarakiBot, [24 мар. 2025 в 14:38]
€1300 2-bedroom apartment to rent
Limassol, Limassol - Neapolis
https://www.bazaraki.com/adv/5669795_2-bedroom-apartment-to-rent/

BazarakiBot, [24 мар. 2025 в 15:04]
€ 1400 ➝ 1300 2-bedroom apartment to rent
Limassol, Limassol - Neapolis
https://www.bazaraki.com/adv/5708036_2-bedroom-apartment-to-rent/
             """

In [22]:
re.findall(r'https://')

In [9]:
pattern = r'https://www\.bazaraki\.com/adv/\d+_[\w-]+/'
links_list = re.findall(pattern, link_text)

In [26]:
len(links_list)

177

In [27]:
len(set(links_list))

155

In [11]:
unique_list = list(set(links_list))

In [12]:
links_df = pd.DataFrame()
links_df['link'] = unique_list

In [13]:
links_df.to_excel(f'../data/unique_links290325.xlsx', index=False)

# Parse Loops

In [43]:
listing_dicts = []
for link in tqdm(unique_list):
    try:
        # Example usage
        url = link
        headers = {'User-Agent': 'Mozilla/5.0'}
        response = requests.get(url, headers=headers)
        listing_data = parse_bazaraki_property(response.text, url)
        listing_dicts.append(listing_data)
        time.sleep(0.4)
    except Exception as e:
        print(f'Exception on {link}: {e}\n')

100%|██████████| 155/155 [15:26<00:00,  5.98s/it]  


In [45]:
listing_dicts[0]

{'Link': 'https://www.bazaraki.com/adv/5664359_2-bedroom-apartment-to-rent/',
 'District': None,
 'CountOfRooms': None,
 'Floor': None,
 'Square': None,
 'Furnishing': None,
 'Parking': None,
 'Balcony': False,
 'AC': None,
 'EnergyEfficiency': None,
 'DistanceToJob': False,
 'DistanceToBeach': False,
 'DoubleGlazedWindows': False,
 'Elevator': False,
 'WiFi': False,
 'SoundProofing': False,
 'Price': None}

In [52]:
prices_df = pd.DataFrame(listing_dicts)

In [53]:
prices_df

,Link,District,CountOfRooms,Floor,Square,Furnishing,Parking,Balcony,AC,EnergyEfficiency,DistanceToJob,DistanceToBeach,DoubleGlazedWindows,Elevator,WiFi,SoundProofing,Price
0,https://www.bazaraki.com/adv/5664359_2-bedroom...,None,None,None,None,None,None,False,None,None,False,False,False,False,False,False,None
1,https://www.bazaraki.com/adv/5593137_2-bedroom...,"Limassol, Ypsonas",2,2nd,2nd,"Full, all rooms",Uncovered,True,"Full, all rooms",A,False,False,False,True,False,False,1.200
2,https://www.bazaraki.com/adv/5693859_2-bedroom...,"Limassol, Limassol - Omonia",2,1st,1st,"Full, all rooms",Covered,True,"Full, all rooms",N/A,False,False,False,True,False,False,1.350
3,https://www.bazaraki.com/adv/5669144_2-bedroom...,"Limassol, Limassol - Agios Nicolaos",1,4th,4th,3105,No,False,"Full, all rooms",N/A,False,False,False,False,False,False,1.350
4,https://www.bazaraki.com/adv/5693505_2-bedroom...,"Limassol, Limassol - Agios Ioannis",1,4th,4th,3065,No,False,"Full, all rooms",A,False,False,False,False,False,False,1.200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,https://www.bazaraki.com/adv/5621114_2-bedroom...,"Limassol, Limassol - Tsirion",1,1st,1st,3081,Covered,True,"Full, all rooms",N/A,False,False,False,True,False,False,1.300
151,https://www.bazaraki.com/adv/5397909_2-bedroom...,"Limassol, Limassol - Omonia",2,1st,1st,3048,Covered,True,"Full, all rooms",N/A,False,False,False,True,False,False,1.350
152,https://www.bazaraki.com/adv/5272205_2-bedroom...,"Limassol, Potamos Germasogeias",1,3rd,3rd,4040,Covered,True,"Full, all rooms",N/A,False,False,False,True,False,False,1.200
153,https://www.bazaraki.com/adv/5700476_2-bedroom...,"Limassol, Limassol - Agios Spyridon",2,1st,1st,"Full, all rooms",Covered,True,"Full, all rooms",A,False,False,False,True,False,False,1.400


In [56]:
prices_df['District'].dropna().shape

(141,)

In [57]:
prices_df['District'].value_counts()

District
Limassol, Limassol - Mesa Geitonia          10
Limassol, Limassol - Zakaki                 10
Limassol, Limassol - Neapolis                8
Limassol, Germasogeia Tourist Area           8
Limassol, Polemidia Kato                     7
Limassol, Limassol - Katholiki               7
Limassol, Limassol - Agios Nicolaos          6
Limassol, Germasogeia                        6
Limassol, Limassol - Omonia                  6
Limassol, Ypsonas                            5
Limassol, Historical Center                  5
Limassol, Limassol - Kapsalos                5
Limassol, Limassol - Agios Spyridon          4
Limassol, Mesa Geitonia                      4
Limassol, Limassol - Apostolos Andreas       4
Limassol, Limassol - Agios Ioannis           3
Limassol, Limassol - Agia Zoni               3
Limassol, Limassol - Agia Fyla               3
Limassol, Mouttagiaka Tourist Area           3
Limassol, Limassol - Agios Georgios          3
Limassol, Limassol - Petrou Kai Pavlou       3
Lima

In [67]:
def extract_price(x):
    if pd.isnull(x):
        return np.nan
    x = str(x).replace('.', '')
    return int(x)

In [68]:
prices_df['price_int'] = prices_df['Price'].apply(extract_price)

In [72]:
prices_df.groupby('District').agg(min_price=('price_int', 'min'),
                                  mean_price=('price_int', 'mean'),
                                  median_price=('price_int', 'median'),
                                  count_of_price=('price_int', 'count'),
                                  max_price=('price_int', 'max')
                                  ).sort_values(by='mean_price', ascending=False)

,min_price,mean_price,median_price,count_of_price,max_price
District,,,,,
"Limassol, Panthea",1750.0,1750.000000,1750.0,1,1750.0
"Limassol, Ekali",1550.0,1550.000000,1550.0,1,1550.0
"Limassol, Limassol - Kapsalos",1200.0,1410.000000,1300.0,5,1650.0
"Limassol, Limassol - Katholiki",1250.0,1400.000000,1400.0,7,1650.0
"Limassol, Limassol - Agia Zoni",1200.0,1383.333333,1350.0,3,1600.0
"Limassol, Limassol - Zakaki",1100.0,1364.000000,1350.0,10,1600.0
"Limassol, Limassol - Agios Spyridon",1300.0,1362.500000,1375.0,4,1400.0
"Limassol, Limassol - Agia Napa",1350.0,1350.000000,1350.0,1,1350.0
"Limassol, Polemidia Kato",1100.0,1350.000000,1350.0,7,1650.0


In [76]:
prices_df.to_excel(f'../data/prices_df240325.xlsx', index=False)

In [64]:
import numpy as np

In [2]:
for script in soup.find_all('script'):
        if script.string and 'coordinates' in script.string:
            match = re.search(r'"coordinates":\s*\[\s*([\d.]+)\s*,\s*([\d.]+)\s*\]', script.string)
            if match:
                lon = float(match.group(1))
                lat = float(match.group(2))
                return {'lat': lat, 'lon': lon}
    return None

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 8)